---
format:
    html:
        embed-resources: true
---

# Cleaning: Part-2 

The goal here is exactly the same as `HW-3.2-cleaning-1.ipynb`, except this time you should repeat the exercise but by leveraging LLM APIs and prompt engineering to stream line the cleaning process. 

Essentially, your job is to write an LLM wrapper to clean the job descriptions. 

How you do this is up to you. You can use any LLM API that you want, and you can use any prompt engineering techniques you want. 

For example, you can wrap around OpenAI's ChatGPT, however this requires that you buy a few credits, e.g. 5 to 10$ (you don't have to if you don't want to, you can use free LLM options as well).

Here is an example of how to use OpenAI's API:

[https://jfh.georgetown.domains/centralized-lecture-content/content/computer-science/general-concepts/openAI-API-example/notes.html](https://jfh.georgetown.domains/centralized-lecture-content/content/computer-science/general-concepts/openAI-API-example/notes.html)

There are also various LLM APIs that you can wrap around to get partial access. Do some googling and find a tool that seems like it will fit your needs.  

* [https://ai.google.dev/gemini-api/docs/quickstart?lang=python](https://ai.google.dev/gemini-api/docs/quickstart?lang=python)

